# Игра "Угадай число"

Сначала импортируем зависимости.

Сгруппируем импорты в порядке, описанном в PEP8. Так у нас
получится две группы: импорты стандартной библиотеки и сторонние.
Между группами импортов по PEP8 оставим пустую строку. Ниже также
будем придерживаться PEP8.

Библиотеку `numpy` будем использовать для генерации
псевдослучайных чисел.

In [5]:
from typing import Callable

import numpy as np

## Функция для угадывания числа

Сначала установим границы интервала поиска, содержащего угадываемое
число.
 
Затем, пока не будет найден ответ, будем предлагать в качестве него
число, максимально близкое к середине этого интервала, которое будет
становиться его новой границей, если это не верная отгадка.
 
В зависимости от того, больше ли наше число, чем отгадка, или меньше,
оно будет становиться, соответственно, новой верхней или нижней
границей интервала поиска.

In [ ]:
def predict_fast(number: int) -> int:
    """
    Угадывает число от 1 до 100 не более чем за 7 попыток.

    Args:
        number (int): Загаданное число.

    Returns:
        int: Количество затраченных попыток угадывания.
    """
    guessing_attempts_count = 0

    # Зададим границы, между которыми будем искать загаданное число.
    lower_boundary = 0
    upper_boundary = 101

    while True:
        guessing_attempts_count += 1

        # Чтобы угадать побыстрее, каждый раз выбираем середину
        # интервала, в котором содержится искомое число, так
        # область поиска сокращается примерно пополам.
        prediction = (lower_boundary + upper_boundary) // 2

        if number == prediction:
            break  # Останавливаем угадывание, поскольку нашли отгадку.

        if prediction > number:
            upper_boundary = prediction
        else:
            lower_boundary = prediction

    return guessing_attempts_count

## Функция для оценки

Эта функция необходима, чтобы определить, за какое число попыток
программа угадывает наше число.

Тут я взял за основу функцию `score_game`, приведённую в [условии
задачи](https://apps.skillfactory.ru/learning/course/course-v1:SkillFactory+MFTIDS+SEP2023/block-v1:SkillFactory+MFTIDS+SEP2023+type@sequential+block@24479890fdb6421b99379be454b66966/block-v1:SkillFactory+MFTIDS+SEP2023+type@vertical+block@7166cc10f25642e890e6d2e98b4297ec) на платформе "Skillfactory". Но поменял функцию, использующуюся
для округления результата с `int` на `round`, и добавил закрепление
состояния генератора псевдослучайных чисел. 

In [9]:
def score_game(predict_fn: Callable[[int], int]) -> int:
    """
    За какое количество попыток в среднем алгоритм угадывает число.
    Среднее рассчитывается по 1000 подходам угадывания псевдослучайных
    чисел (для воспроизводимости используется фиксированный генератор).

    Args:
        predict_fn (Callable[[int], int]): Оцениваемая функция
            угадывания, которая должна принимать загаданное число
            в качестве аргумента, а возвращать количество затраченных
            на его угадывание попыток.

    Returns:
        int: Округлённое среднее количество попыток угадывания.
    """
    # Количество попыток угадывания, которое потребовалось
    # во время каждой игры будем добавлять в список для последующих
    # расчётов статистики.
    attempts_counts = []
    
    # Загадываем числа
    random_state = np.random.RandomState(1)
    random_numbers = random_state.randint(1, 101, size=1000)

    # Играем, записываем количества попыток для последующей оценки.
    for number in random_numbers:
        attempts_counts.append(predict_fn(number))

    game_score = round(np.mean(attempts_counts))
    return game_score

# Оценка качества алгоритма

Теперь выполним наши функции для получения вышеупомянутой оценки
среднего количества попыток угадывания, которые требуются нашему
алгоритму. 

In [10]:
score = score_game(predict_fast)
print(f"Наш алгоритм угадывает число в среднем за {score} попыток.")

Наш алгоритм угадывает число в среднем за 6 попыток.
